# Chess.com Datenabzug
### 23.12.2022
Download meiner Daten aus Chess.com über die [Chess.com API](https://www.chess.com/news/view/published-data-api) und Zusammenfassung als einzelne JSON-Datei

In [18]:
import pandas as pd
import chess.pgn
import urllib
import urllib.request
import json
import chess.pgn
import re
import sys
import os.path
import pathlib
import logging
from datetime import datetime
import sys, traceback

### Löschen möglicher alter PGN-Dateien

In [19]:
for f in os.listdir('data'):
    os.remove(os.path.join('data', f))

### Download monatliche Stände

In [20]:
# Use of the api to download monthly archives

username = "oezguen" 
baseUrl = "https://api.chess.com/pub/player/" + username + "/games/"
archivesUrl = baseUrl + "archives"

# read the archives url and store in a list
f = urllib.request.urlopen(archivesUrl)
archives = f.read().decode("utf-8")
archives = archives.replace("{\"archives\":[\"", "\",\"")
archivesList = archives.split("\",\"" + baseUrl)
archivesList[len(archivesList)-1] = archivesList[len(archivesList)-1].rstrip("\"]}")

# download all the archives
for i in range(len(archivesList)-1):
    url = baseUrl + archivesList[i+1] + "/pgn"
    filename = archivesList[i+1].replace("/", "-")
    urllib.request.urlretrieve(url, "data/" + filename + ".pgn")
    print(filename + ".pgn wird heruntergeladen.", end="\r")
print ("Alle Dateien wurden heruntergeladen.")

Alle Dateien wurden heruntergeladen.


### Zusammenfassung als eine JSON-Datei

In [21]:

log = logging.getLogger().error
inp_dir = pathlib.Path('data/')
out_dir = pathlib.Path('')

def get_file_list(local_path):
    tree = os.walk(str(local_path))
    file_list = []
    out = []
    test = r'.+pgn$'
    for i in tree:
        file_list = i[2]

    for name in file_list:
        if(len(re.findall(test, name))):
            out.append(str(local_path / name))
    return out


def get_data(pgn_file):
    node = chess.pgn.read_game(pgn_file)
    while node is not None:
        data = node.headers
        data["moves"] = []
        while node.variations:
            next_node = node.variation(0)
            data["moves"].append(
                    re.sub("\{.*?\}", "", node.board().san(next_node.move)))
            node = next_node
        node = chess.pgn.read_game(pgn_file)
        out_dict = {}
        for key in data.keys():
            out_dict[key] = data.get(key)
        yield out_dict


def convert_file(file_path):
    file_name = file_path.name.replace(file_path.suffix, '') + '.json'
    log('convert file '+file_path.name)
    out_list = []
    try:
        json_file = open(str(out_dir / file_name), 'w')
        pgn_file = open(str(file_path), encoding='ISO-8859-1')
        for count_d, data in enumerate(get_data(pgn_file), start=0):
            log(file_path.name+' '+str(count_d))
            out_list.append(data)

        log(' save '+file_path.name)
        json.dump(out_list, json_file)
        json_file.close()
        log('done')
    except Exception as e:
        log(traceback.format_exc(10))
        log('ERROR file '+file_name+' not converted')


def create_join_file(file_list):
    name = str(out_dir / 'chess_data.json')
    open(name, 'w').close()
    json_file = open(str(out_dir / 'my_chess_data.json'), 'a')
    json_file.write('[')
    for count_f, file in enumerate(file_list, start=0):
        pgn_file = open(file, encoding='ISO-8859-1')
        for count_d, data in enumerate(get_data(pgn_file), start=0):
            print(str(count_f)+' '+str(count_d), end="\r")
            if count_f or count_d:
                json_file.write(',')
            data_str = json.dumps(data)
            json_file.write(data_str)
        print(pathlib.Path(file).name, end="\r")
    json_file.write(']')
    json_file.close()


file_list = get_file_list(inp_dir)
file_list.sort()
create_join_file(file_list)


### Löschen der PGN-Dateien

In [22]:
for f in os.listdir('data'):
    os.remove(os.path.join('data', f))